In [ ]:
# 禁用 GPU，强制使用 CPU（必须在 import tensorflow/keras 之前）
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# 导入所有必需的库
import numpy as np
import pickle
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Conv1D, GlobalMaxPooling1D
from keras.layers import Dropout, Multiply, Permute, Lambda, Flatten, Concatenate
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras import backend as K
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# 加载特征的辅助函数
def load_features(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

print("库导入完成")


In [ ]:
# ============================================
# 加载所有特征数据
# ============================================
print("=" * 50)
print("加载特征数据")
print("=" * 50)

# 加载特征
w2v_cmbs = load_features('./data2025/fold2_textual_wav2vec.pkl')
bert_embs = load_features('./data2025/fold2_textual_bert.pkl')
baidu_embs = load_features('./data2025/fold2_textual_baidu.pkl')
label = load_features('./data2025/fold2_labels.pkl')

# 打印特征维度
print(f"Wav2vec 特征 (train): {np.asarray(w2v_cmbs['train']).shape}")
print(f"BERT 特征 (train): {np.asarray(bert_embs['train']).shape}")
print(f"Baidu 特征 (train): {np.asarray(baidu_embs['train']).shape}")
print(f"标签 (train): {np.asarray(label['train']).shape}")
print("\n数据加载完成")


## 数据统计 + 类别权重计算

In [36]:
# ============================================
# 改进版模型 - 添加正则化和类别平衡
# ============================================

# 额外导入
from keras.layers import Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight

# 查看数据分布
print("=" * 50)
print("数据集统计信息")
print("=" * 50)
train_labels = np.array(label['train'])
test_labels = np.array(label['test'])

print(f"训练集大小: {len(train_labels)}")
print(f"测试集大小: {len(test_labels)}")
print(f"\n训练集类别分布:")
for i, name in enumerate(['bored', 'happy', 'interested', 'tired', 'confusion']):
    count = np.sum(train_labels == i)
    print(f"  {name}: {count} ({count/len(train_labels)*100:.1f}%)")

print(f"\n测试集类别分布:")
for i, name in enumerate(['bored', 'happy', 'interested', 'tired', 'confusion']):
    count = np.sum(test_labels == i)
    print(f"  {name}: {count} ({count/len(test_labels)*100:.1f}%)")

# 计算类别权重
class_weights = compute_class_weight('balanced', 
                                     classes=np.unique(train_labels), 
                                     y=train_labels)
class_weight_dict = dict(enumerate(class_weights))
print(f"\n类别权重: {class_weight_dict}")

数据集统计信息
训练集大小: 346
测试集大小: 176

训练集类别分布:
  bored: 94 (27.2%)
  happy: 58 (16.8%)
  interested: 63 (18.2%)
  tired: 74 (21.4%)
  confusion: 57 (16.5%)

测试集类别分布:
  bored: 49 (27.8%)
  happy: 28 (15.9%)
  interested: 33 (18.8%)
  tired: 37 (21.0%)
  confusion: 29 (16.5%)

类别权重: {0: 0.7361702127659574, 1: 1.193103448275862, 2: 1.0984126984126985, 3: 0.9351351351351351, 4: 1.2140350877192982}


## TextCNN 模型定义函数

In [97]:
# ============================================
# TextCNN 模型定义函数
# ============================================
def build_textcnn(input_shape, num_classes=5, kernel_sizes=[3, 4, 5], num_filters=128, dropout_rate=0.4):
    """
    构建 TextCNN 模型
    Args:
        input_shape: 输入形状 (seq_len, embedding_dim)
        num_classes: 分类数量
        kernel_sizes: 卷积核大小列表
        num_filters: 每种卷积核的过滤器数量
        dropout_rate: Dropout 比率
    """
    x = Input(input_shape)
    
    # 输入 Dropout
    h = Dropout(0.2)(x)
    
    # 多尺度卷积
    convs = []
    for kernel_size in kernel_sizes:
        c = Conv1D(num_filters, kernel_size, activation='relu', padding='same',
                   kernel_regularizer=l2(0.01))(h)
        c = GlobalMaxPooling1D()(c)
        convs.append(c)
    
    # 拼接多尺度特征
    if len(convs) > 1:
        fea_cnn = Concatenate()(convs)
    else:
        fea_cnn = convs[0]
    
    # Dropout
    fea_cnn_dropout = Dropout(dropout_rate)(fea_cnn)
    
    # 全连接层
    fea_dense = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(fea_cnn_dropout)
    fea_dense = Dropout(0.3)(fea_dense)
    
    # 输出层
    output = Dense(num_classes, activation='softmax')(fea_dense)
    
    model = Model(inputs=x, outputs=output)
    return model

print("TextCNN 模型定义完成")

TextCNN 模型定义完成


## Attention 机制定义

In [ ]:
# ============================================
# Attention 机制定义
# ============================================
def attention_layer(inputs):
    """
    Self-Attention 层
    Args:
        inputs: (batch, time_steps, features)
    Returns:
        attended: (batch, time_steps, features)
    """
    # 计算注意力权重
    attention_weights = Dense(1, activation='tanh')(inputs)
    attention_weights = Flatten()(attention_weights)
    attention_weights = keras.layers.Activation('softmax')(attention_weights)
    attention_weights = keras.layers.RepeatVector(inputs.shape[-1])(attention_weights)
    attention_weights = Permute([2, 1])(attention_weights)
    
    # 应用注意力
    attended = Multiply()([inputs, attention_weights])
    return attended

print("Attention 机制定义完成")


# Wav2vec 特征实验

## 1. TextCNN + Wav2vec

In [125]:
# ============================================
# TextCNN + Wav2vec 特征
# ============================================
print("\n" + "=" * 50)
print("TextCNN + Wav2vec 特征")
print("=" * 50)

# 获取输入形状
w2v_train = np.asarray(w2v_cmbs['train'])
w2v_test = np.asarray(w2v_cmbs['test'])
input_shape_w2v = w2v_train.shape[1:]  # (82, 100)
print(f"Wav2vec 输入形状: {input_shape_w2v}")

# 构建模型
model_cnn_w2v = build_textcnn(input_shape_w2v, num_classes=5)
model_cnn_w2v.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['acc']
)
model_cnn_w2v.summary()

# 回调函数
callback_list = [
    ModelCheckpoint(
        filepath="T_TextCNN_wav2vec.tf",
        monitor='val_loss',
        save_best_only=True,
        save_freq='epoch'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
]

# 训练
history_cnn_w2v = model_cnn_w2v.fit(
    x=w2v_train,
    y=np.asarray(label['train']),
    batch_size=16,
    epochs=50,
    validation_data=[w2v_test, np.asarray(label['test'])],
    callbacks=callback_list,
    class_weight=class_weight_dict,
    verbose=1
)

# 测试
model_cnn_w2v = keras.models.load_model("T_TextCNN_wav2vec.tf")
pred = model_cnn_w2v.predict(w2v_test)
predicted_labels = pred.argmax(axis=1)
numeric_labels = np.array(label['test'])

eval_res = classification_report(
    numeric_labels, predicted_labels,
    target_names=['bored', 'happy', 'interested', 'tired', 'confusion'],
    digits=4, output_dict=False, zero_division=0
)
print(eval_res)
print('TextCNN + Wav2vec:')
cm = confusion_matrix(y_true=numeric_labels.tolist(), y_pred=predicted_labels.tolist())
print(cm)


TextCNN + Wav2vec 特征
Wav2vec 输入形状: (82, 100)
Model: "model_75"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_83 (InputLayer)          [(None, 82, 100)]    0           []                               
                                                                                                  
 dropout_246 (Dropout)          (None, 82, 100)      0           ['input_83[0][0]']               
                                                                                                  
 conv1d_96 (Conv1D)             (None, 82, 128)      38528       ['dropout_246[0][0]']            
                                                                                                  
 conv1d_97 (Conv1D)             (None, 82, 128)      51328       ['dropout_246[0][0]']            
                                             

INFO:tensorflow:Assets written to: ./data2025/res2/T_TextCNN_wav2vec.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/T_TextCNN_wav2vec.tf/assets


22/22 [==============================] - 1s 36ms/step - loss: 6.7598 - acc: 0.2216 - val_loss: 6.2520 - val_acc: 0.2905
Epoch 2/50
14/22 [==================>...........] - ETA: 0s - loss: 6.0492 - acc: 0.2857

INFO:tensorflow:Assets written to: ./data2025/res2/T_TextCNN_wav2vec.tf/assets


...

INFO:tensorflow:Assets written to: ./data2025/res2/T_TextCNN_wav2vec.tf/assets


22/22 [==============================] - 1s 33ms/step - loss: 0.7931 - acc: 0.9417 - val_loss: 1.7374 - val_acc: 0.5140
Epoch 43/50
22/22 [==============================] - 0s 6ms/step - loss: 0.7647 - acc: 0.9359 - val_loss: 1.8357 - val_acc: 0.4916
Epoch 44/50
22/22 [==============================] - 0s 5ms/step - loss: 0.7466 - acc: 0.9534 - val_loss: 1.7694 - val_acc: 0.5140
Epoch 45/50
22/22 [==============================] - 0s 6ms/step - loss: 0.7276 - acc: 0.9446 - val_loss: 1.7838 - val_acc: 0.5140
Epoch 46/50
22/22 [==============================] - 0s 6ms/step - loss: 0.7832 - acc: 0.9009 - val_loss: 1.8055 - val_acc: 0.5028
Epoch 47/50
22/22 [==============================] - 0s 6ms/step - loss: 0.7479 - acc: 0.9504 - val_loss: 1.7596 - val_acc: 0.4972
Epoch 48/50
22/22 [==============================] - 0s 5ms/step - loss: 0.7132 - acc: 0.9621 - val_loss: 1.7860 - val_acc: 0.5084
Epoch 49/50
22/22 [==============================] - 0s 6ms/step - loss: 0.7061 - acc: 0.9329 

INFO:tensorflow:Assets written to: ./data2025/res2/T_TextCNN_wav2vec.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/T_TextCNN_wav2vec.tf/assets


6/6 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

       bored     0.4615    0.4800    0.4706        50
       happy     0.4103    0.5926    0.4848        27
  interested     0.4000    0.3784    0.3889        37
       tired     0.6296    0.4474    0.5231        38
   confusion     0.5385    0.5185    0.5283        27

    accuracy                         0.4749       179
   macro avg     0.4880    0.4834    0.4791       179
weighted avg     0.4884    0.4749    0.4757       179

TextCNN + Wav2vec:
[[24  7  6  9  4]
 [ 2 16  9  0  0]
 [ 7 10 14  0  6]
 [16  1  2 17  2]
 [ 3  5  4  1 14]]


## 2. LSTM + Wav2vec

In [127]:
# ============================================
# ziBiLstm 风格 LSTM + Wav2vec 特征
# ============================================
print("\n" + "=" * 50)
print("ziBiLstm风格 LSTM + Wav2vec 特征")
print("=" * 50)

input_shape_w2v = np.asarray(w2v_cmbs['train']).shape[1:]
print(f"Wav2vec 输入形状: {input_shape_w2v}")

# 构建模型
model_zilstm_w2v = build_lstm_pooling(input_shape_w2v)
model_zilstm_w2v.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['acc']
)
model_zilstm_w2v.summary()

# 回调函数
callback_list = [
    ModelCheckpoint(
        filepath="T_ziLSTM_wav2vec.tf",
        monitor='val_loss',
        save_best_only=True,
        save_freq='epoch'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
]

# 训练
history_zilstm_w2v = model_zilstm_w2v.fit(
    x=np.asarray(w2v_cmbs['train']),
    y=np.asarray(label['train']),
    batch_size=16,
    epochs=50,
    validation_data=[np.asarray(w2v_cmbs['test']), np.asarray(label['test'])],
    callbacks=callback_list,
    class_weight=class_weight_dict,
    verbose=1
)

# 测试
model_zilstm_w2v = keras.models.load_model("T_ziLSTM_wav2vec.tf")
pred = model_zilstm_w2v.predict(np.asarray(w2v_cmbs['test']))
predicted_labels = pred.argmax(axis=1)
numeric_labels = np.array(label['test'])

eval_res = classification_report(
    numeric_labels, predicted_labels,
    target_names=['bored', 'happy', 'interested', 'tired', 'confusion'],
    digits=4, output_dict=False, zero_division=0
)
print(eval_res)
print('ziBiLstm风格 LSTM + Wav2vec:')
cm = confusion_matrix(y_true=numeric_labels.tolist(), y_pred=predicted_labels.tolist())
print(cm)


ziBiLstm风格 LSTM + Wav2vec 特征
Wav2vec 输入形状: (82, 100)
Model: "model_77"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_85 (InputLayer)       [(None, 82, 100)]         0         
                                                                 
 dropout_252 (Dropout)       (None, 82, 100)           0         
                                                                 
 lstm_44 (LSTM)              (None, 82, 128)           117248    
                                                                 
 global_max_pooling1d_143 (G  (None, 128)              0         
 lobalMaxPooling1D)                                              
                                                                 
 dropout_253 (Dropout)       (None, 128)               0         
                                                                 
 dense_154 (Dense)           (None, 128)               16512     
    

INFO:tensorflow:Assets written to: ./data2025/res2/T_ziLSTM_wav2vec.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/T_ziLSTM_wav2vec.tf/assets


22/22 [==============================] - 3s 106ms/step - loss: 5.0018 - acc: 0.2245 - val_loss: 4.1516 - val_acc: 0.3855
Epoch 2/50
20/22 [==========================>...] - ETA: 0s - loss: 3.6577 - acc: 0.2969

INFO:tensorflow:Assets written to: ./data2025/res2/T_ziLSTM_wav2vec.tf/assets


...

INFO:tensorflow:Assets written to: ./data2025/res2/T_ziLSTM_wav2vec.tf/assets


22/22 [==============================] - 2s 97ms/step - loss: 1.1437 - acc: 0.5889 - val_loss: 1.4518 - val_acc: 0.4358
Epoch 32/50
20/22 [==========================>...] - ETA: 0s - loss: 1.1712 - acc: 0.5750

INFO:tensorflow:Assets written to: ./data2025/res2/T_ziLSTM_wav2vec.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/T_ziLSTM_wav2vec.tf/assets


22/22 [==============================] - 2s 95ms/step - loss: 1.1713 - acc: 0.5831 - val_loss: 1.4292 - val_acc: 0.4637
Epoch 33/50
22/22 [==============================] - 0s 22ms/step - loss: 1.1484 - acc: 0.6152 - val_loss: 1.4649 - val_acc: 0.4637
Epoch 34/50
22/22 [==============================] - 0s 22ms/step - loss: 1.1380 - acc: 0.6152 - val_loss: 1.5245 - val_acc: 0.3911
Epoch 35/50
22/22 [==============================] - 0s 22ms/step - loss: 1.1332 - acc: 0.5948 - val_loss: 1.4740 - val_acc: 0.4525
Epoch 36/50
22/22 [==============================] - 1s 23ms/step - loss: 1.1594 - acc: 0.5743 - val_loss: 1.4955 - val_acc: 0.4358
Epoch 37/50
22/22 [==============================] - 0s 22ms/step - loss: 1.1360 - acc: 0.6093 - val_loss: 1.4302 - val_acc: 0.4525
Epoch 38/50
22/22 [==============================] - 0s 21ms/step - loss: 1.1641 - acc: 0.6006 - val_loss: 1.6238 - val_acc: 0.4134
Epoch 39/50
22/22 [==============================] - 0s 20ms/step - loss: 1.1726 - acc: 

## 3. Attention + BiLSTM + Wav2vec

In [132]:
# ============================================
# Attention + BiLSTM + Wav2vec 特征 (ziLSTM风格 + 正则化)
# ============================================
print("\n" + "=" * 50)
print("Attention + BiLSTM + Wav2vec 特征 (ziLSTM风格 + 正则化)")
print("=" * 50)

# 获取输入形状
w2v_train = np.asarray(w2v_cmbs['train'])
w2v_test = np.asarray(w2v_cmbs['test'])
input_shape_w2v = w2v_train.shape[1:]
print(f"Wav2vec 输入形状: {input_shape_w2v}")

# 构建模型
x = Input(input_shape_w2v)
h = Dropout(0.2)(x)  # 输入层 Dropout
h = Attention()([h, h])
h = Dropout(0.3)(h)  # Attention 后 Dropout
# ziLSTM 风格: BiLSTM(return_sequences=True) + GlobalMaxPooling1D
h = Bidirectional(LSTM(64, return_sequences=True, return_state=False,
                       kernel_regularizer=l2(0.01),
                       recurrent_regularizer=l2(0.01)))(h)
h = GlobalMaxPooling1D()(h)  # 提取最显著特征
h = Dropout(0.4)(h)
h = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(h)
h = Dropout(0.3)(h)
res = Dense(5, 'softmax', kernel_regularizer=l2(0.01))(h)

model_att_bilstm_w2v = Model(inputs=x, outputs=res)
model_att_bilstm_w2v.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['acc']
)
model_att_bilstm_w2v.summary()

# 回调函数
callback_list = [
    ModelCheckpoint(
        filepath="T_Att_BiLSTM_wav2vec.tf",
        monitor='val_acc',
        save_best_only=True,
        save_freq='epoch'
    ),
    EarlyStopping(
        monitor='val_acc',
        patience=10,
        restore_best_weights=True,
        mode='max',
        verbose=1
    )
]

# 训练
history_att_bilstm_w2v = model_att_bilstm_w2v.fit(
    x=w2v_train,
    y=np.asarray(label['train']),
    batch_size=16,
    epochs=50,
    validation_data=[w2v_test, np.asarray(label['test'])],
    callbacks=callback_list,
    class_weight=class_weight_dict,
    verbose=1
)

# 输出最佳准确度
acc_key = 'acc' if 'acc' in history_att_bilstm_w2v.history else 'accuracy'
val_acc_key = 'val_acc' if 'val_acc' in history_att_bilstm_w2v.history else 'val_accuracy'

best_train_acc = max(history_att_bilstm_w2v.history[acc_key])
best_val_acc = max(history_att_bilstm_w2v.history[val_acc_key])
best_epoch = history_att_bilstm_w2v.history[val_acc_key].index(best_val_acc) + 1

print("\n" + "="*60)
print("训练结果总结")
print("="*60)
print(f"最佳训练准确度 (Train ACC): {best_train_acc:.4f}")
print(f"最佳验证准确度 (Val ACC): {best_val_acc:.4f}")
print(f"最佳准确度出现在第 {best_epoch} 个epoch")
print("="*60)

# 测试
model_att_bilstm_w2v = keras.models.load_model("T_Att_BiLSTM_wav2vec.tf")
pred = model_att_bilstm_w2v.predict(w2v_test)
predicted_labels = pred.argmax(axis=1)
numeric_labels = np.array(label['test'])

eval_res = classification_report(
    numeric_labels, predicted_labels,
    target_names=['bored', 'happy', 'interested', 'tired', 'confusion'],
    digits=4, output_dict=False, zero_division=0
)
print(eval_res)
print('Attention + BiLSTM + Wav2vec (ziLSTM风格 + 正则化):')
cm = confusion_matrix(y_true=numeric_labels.tolist(), y_pred=predicted_labels.tolist())
print(cm)


Attention + BiLSTM + Wav2vec 特征 (ziLSTM风格 + 正则化)
Wav2vec 输入形状: (82, 100)
Model: "model_82"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_90 (InputLayer)          [(None, 82, 100)]    0           []                               
                                                                                                  
 dropout_271 (Dropout)          (None, 82, 100)      0           ['input_90[0][0]']               
                                                                                                  
 attention_24 (Attention)       (None, 82, 100)      0           ['dropout_271[0][0]',            
                                                                  'dropout_271[0][0]']            
                                                                                                  
 dropout_272 (Dro

INFO:tensorflow:Assets written to: ./data2025/res2/1T_Att_BiLSTM_wav2vec.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/1T_Att_BiLSTM_wav2vec.tf/assets


22/22 [==============================] - 8s 252ms/step - loss: 6.6909 - acc: 0.2099 - val_loss: 6.1685 - val_acc: 0.3520
Epoch 2/50
22/22 [==============================] - ETA: 0s - loss: 5.7863 - acc: 0.2624

INFO:tensorflow:Assets written to: ./data2025/res2/1T_Att_BiLSTM_wav2vec.tf/assets


...

INFO:tensorflow:Assets written to: ./data2025/res2/1T_Att_BiLSTM_wav2vec.tf/assets


22/22 [==============================] - 5s 228ms/step - loss: 5.0707 - acc: 0.2770 - val_loss: 4.7115 - val_acc: 0.4693
Epoch 4/50
22/22 [==============================] - ETA: 0s - loss: 4.4611 - acc: 0.3294

INFO:tensorflow:Assets written to: ./data2025/res2/1T_Att_BiLSTM_wav2vec.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/1T_Att_BiLSTM_wav2vec.tf/assets


22/22 [==============================] - 6s 271ms/step - loss: 4.4611 - acc: 0.3294 - val_loss: 4.1720 - val_acc: 0.4972
Epoch 5/50
22/22 [==============================] - 1s 24ms/step - loss: 3.9606 - acc: 0.3411 - val_loss: 3.7217 - val_acc: 0.4413
Epoch 6/50
22/22 [==============================] - 1s 26ms/step - loss: 3.5111 - acc: 0.3994 - val_loss: 3.3338 - val_acc: 0.4022
Epoch 7/50
22/22 [==============================] - 1s 25ms/step - loss: 3.1686 - acc: 0.4286 - val_loss: 3.0098 - val_acc: 0.3631
Epoch 8/50
22/22 [==============================] - 1s 24ms/step - loss: 2.8821 - acc: 0.3848 - val_loss: 2.7832 - val_acc: 0.3966
Epoch 9/50
22/22 [==============================] - 1s 25ms/step - loss: 2.6326 - acc: 0.4257 - val_loss: 2.5955 - val_acc: 0.4525
Epoch 10/50
22/22 [==============================] - 1s 24ms/step - loss: 2.4855 - acc: 0.4402 - val_loss: 2.4439 - val_acc: 0.4469
Epoch 11/50
22/22 [==============================] - 1s 26ms/step - loss: 2.3470 - acc: 0.41

## Wav2vec 系列结果对比

In [ ]:
# ============================================
# Wav2vec 特征模型结果对比可视化
# ============================================
import keras

print("\n" + "=" * 50)
print("Wav2vec 特征模型评估")
print("=" * 50)

# 加载模型
model_textcnn = keras.models.load_model('data2025/res2/1T_TextCNN_wav2vec.tf')
model_lstm = keras.models.load_model('data2025/res/1T_LSTM_wav2vec.tf')
model_att_bilstm = keras.models.load_model('data2025/res/1T_Att_BiLSTM_wav2vec.tf')

# 准备测试数据
w2v_test = np.asarray(w2v_cmbs['test'])
y_test = np.asarray(label['test'])

# 评估
test_loss_textcnn, test_acc_textcnn = model_textcnn.evaluate(w2v_test, y_test, verbose=0)
test_loss_lstm, test_acc_lstm = model_lstm.evaluate(w2v_test, y_test, verbose=0)
test_loss_att, test_acc_att = model_att_bilstm.evaluate(w2v_test, y_test, verbose=0)

# 可视化对比
models_names = ['TextCNN', 'LSTM', 'Attention+BiLSTM']
accuracies = [test_acc_textcnn, test_acc_lstm, test_acc_att]

plt.figure(figsize=(10, 6))
plt.bar(models_names, accuracies, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.xlabel('Model', fontsize=12)
plt.ylabel('Test Accuracy', fontsize=12)
plt.title('Wav2vec Feature Models Comparison', fontsize=14, fontweight='bold')
plt.ylim([0, 1])
for i, v in enumerate(accuracies):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom', fontsize=10)
plt.tight_layout()
plt.show()

print("\nWav2vec 特征模型结果：")
print(f"TextCNN: Test Accuracy = {test_acc_textcnn:.4f}, Test Loss = {test_loss_textcnn:.4f}")
print(f"LSTM: Test Accuracy = {test_acc_lstm:.4f}, Test Loss = {test_loss_lstm:.4f}")
print(f"Attention+BiLSTM: Test Accuracy = {test_acc_att:.4f}, Test Loss = {test_loss_att:.4f}")


2025-12-23 22:06:27.606280: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-23 22:06:30.490523: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-23 22:06:31.471643: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-23 22:06:37.509012: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li


Wav2vec 特征模型评估


# BERT-Chinese 特征实验

## 1. TextCNN + BERT

In [91]:
# ============================================
# TextCNN + BERT 特征
# ============================================
print("\n" + "=" * 50)
print("TextCNN + BERT 特征")
print("=" * 50)

# 获取输入形状
bert_train = np.asarray(bert_embs['train'])
bert_test = np.asarray(bert_embs['test'])
input_shape_bert = bert_train.shape[1:]  # (82, 768)
print(f"BERT 输入形状: {input_shape_bert}")

# 构建模型
model_cnn_bert = build_textcnn(input_shape_bert, num_classes=5)
model_cnn_bert.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['acc']
)
model_cnn_bert.summary()

# 回调函数
callback_list = [
    ModelCheckpoint(
        filepath="res2/T_TextCNN_bert.tf",
        monitor='val_loss',
        save_best_only=True,
        save_freq='epoch'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
]

# 训练
history_cnn_bert = model_cnn_bert.fit(
    x=bert_train,
    y=np.asarray(label['train']),
    batch_size=16,
    epochs=50,
    validation_data=[bert_test, np.asarray(label['test'])],
    callbacks=callback_list,
    class_weight=class_weight_dict,
    verbose=1
)

# 测试
model_cnn_bert = keras.models.load_model("res2/T_TextCNN_bert.tf")
pred = model_cnn_bert.predict(bert_test)
predicted_labels = pred.argmax(axis=1)
numeric_labels = np.array(label['test'])

eval_res = classification_report(
    numeric_labels, predicted_labels,
    target_names=['bored', 'happy', 'interested', 'tired', 'confusion'],
    digits=4, output_dict=False, zero_division=0
)
print(eval_res)
print('TextCNN + BERT:')
cm = confusion_matrix(y_true=numeric_labels.tolist(), y_pred=predicted_labels.tolist())
print(cm)


TextCNN + BERT 特征
BERT 输入形状: (82, 768)
Model: "model_46"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_54 (InputLayer)          [(None, 82, 768)]    0           []                               
                                                                                                  
 dropout_149 (Dropout)          (None, 82, 768)      0           ['input_54[0][0]']               
                                                                                                  
 conv1d_57 (Conv1D)             (None, 82, 128)      295040      ['dropout_149[0][0]']            
                                                                                                  
 conv1d_58 (Conv1D)             (None, 82, 128)      393344      ['dropout_149[0][0]']            
                                                   

INFO:tensorflow:Assets written to: ./data2025/res2/1T_TextCNN_bert.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/1T_TextCNN_bert.tf/assets


22/22 [==============================] - 1s 53ms/step - loss: 11.8774 - acc: 0.2711 - val_loss: 9.9168 - val_acc: 0.3240
Epoch 2/50
20/22 [==========================>...] - ETA: 0s - loss: 10.1285 - acc: 0.3469

INFO:tensorflow:Assets written to: ./data2025/res2/1T_TextCNN_bert.tf/assets


...

INFO:tensorflow:Assets written to: ./data2025/res2/1T_TextCNN_bert.tf/assets


22/22 [==============================] - 1s 49ms/step - loss: 1.2935 - acc: 0.9650 - val_loss: 2.2176 - val_acc: 0.6425
Epoch 50/50
20/22 [==========================>...] - ETA: 0s - loss: 1.2660 - acc: 0.9594

INFO:tensorflow:Assets written to: ./data2025/res2/1T_TextCNN_bert.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/1T_TextCNN_bert.tf/assets


6/6 [==============================] - 0s 12ms/step
              precision    recall  f1-score   support

       bored     0.7059    0.7200    0.7129        50
       happy     0.7368    0.5185    0.6087        27
  interested     0.5745    0.7297    0.6429        37
       tired     0.8710    0.7105    0.7826        38
   confusion     0.6129    0.7037    0.6552        27

    accuracy                         0.6872       179
   macro avg     0.7002    0.6765    0.6804       179
weighted avg     0.7044    0.6872    0.6888       179

TextCNN + BERT:
[[36  0  7  4  3]
 [ 2 14  9  0  2]
 [ 1  4 27  0  5]
 [ 8  0  1 27  2]
 [ 4  1  3  0 19]]


## 2. LSTM + BERT

In [92]:
# ============================================
# ziBiLstm 风格 LSTM + BERT 特征
# ============================================
print("\n" + "=" * 50)
print("ziBiLstm风格 LSTM + BERT 特征")
print("=" * 50)

input_shape_bert = np.asarray(bert_embs['train']).shape[1:]
print(f"BERT 输入形状: {input_shape_bert}")

# 构建模型
model_zilstm_bert = build_lstm_pooling(input_shape_bert)
model_zilstm_bert.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['acc']
)
model_zilstm_bert.summary()

# 回调函数
callback_list = [
    ModelCheckpoint(
        filepath="res2/T_ziLSTM_bert.tf",
        monitor='val_loss',
        save_best_only=True,
        save_freq='epoch'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
]

# 训练
history_zilstm_bert = model_zilstm_bert.fit(
    x=np.asarray(bert_embs['train']),
    y=np.asarray(label['train']),
    batch_size=16,
    epochs=50,
    validation_data=[np.asarray(bert_embs['test']), np.asarray(label['test'])],
    callbacks=callback_list,
    class_weight=class_weight_dict,
    verbose=1
)

# 测试
model_zilstm_bert = keras.models.load_model("res2/T_ziLSTM_bert.tf")
pred = model_zilstm_bert.predict(np.asarray(bert_embs['test']))
predicted_labels = pred.argmax(axis=1)
numeric_labels = np.array(label['test'])

eval_res = classification_report(
    numeric_labels, predicted_labels,
    target_names=['bored', 'happy', 'interested', 'tired', 'confusion'],
    digits=4, output_dict=False, zero_division=0
)
print(eval_res)
print('ziBiLstm风格 LSTM + BERT:')
cm = confusion_matrix(y_true=numeric_labels.tolist(), y_pred=predicted_labels.tolist())
print(cm)


ziBiLstm风格 LSTM + BERT 特征
BERT 输入形状: (82, 768)
Model: "model_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_55 (InputLayer)       [(None, 82, 768)]         0         
                                                                 
 dropout_152 (Dropout)       (None, 82, 768)           0         
                                                                 
 lstm_27 (LSTM)              (None, 82, 128)           459264    
                                                                 
 global_max_pooling1d_87 (Gl  (None, 128)              0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_153 (Dropout)       (None, 128)               0         
                                                                 
 dense_94 (Dense)            (None, 128)               16512     
          

INFO:tensorflow:Assets written to: ./data2025/res2/T_ziLSTM_bert.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/T_ziLSTM_bert.tf/assets


22/22 [==============================] - 3s 117ms/step - loss: 9.1395 - acc: 0.2070 - val_loss: 7.7261 - val_acc: 0.2123
Epoch 2/50
20/22 [==========================>...] - ETA: 0s - loss: 6.8762 - acc: 0.2750

INFO:tensorflow:Assets written to: ./data2025/res2/T_ziLSTM_bert.tf/assets


...

INFO:tensorflow:Assets written to: ./data2025/res2/T_ziLSTM_bert.tf/assets


22/22 [==============================] - 3s 157ms/step - loss: 0.7519 - acc: 0.9796 - val_loss: 1.6625 - val_acc: 0.6592
Epoch 26/50
22/22 [==============================] - 1s 29ms/step - loss: 0.6936 - acc: 0.9738 - val_loss: 1.7048 - val_acc: 0.6257
Epoch 27/50
22/22 [==============================] - 1s 32ms/step - loss: 0.6940 - acc: 0.9621 - val_loss: 1.6876 - val_acc: 0.6201
Epoch 28/50
21/22 [===========================>..] - ETA: 0s - loss: 0.6664 - acc: 0.9702

INFO:tensorflow:Assets written to: ./data2025/res2/T_ziLSTM_bert.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/T_ziLSTM_bert.tf/assets


22/22 [==============================] - 2s 114ms/step - loss: 0.6645 - acc: 0.9708 - val_loss: 1.6411 - val_acc: 0.6257
Epoch 29/50
22/22 [==============================] - 1s 32ms/step - loss: 0.6454 - acc: 0.9679 - val_loss: 1.7673 - val_acc: 0.6257
Epoch 30/50
22/22 [==============================] - 1s 32ms/step - loss: 0.7352 - acc: 0.9242 - val_loss: 1.6914 - val_acc: 0.5754
Epoch 31/50
22/22 [==============================] - 1s 33ms/step - loss: 0.7134 - acc: 0.9359 - val_loss: 1.7418 - val_acc: 0.5978
Epoch 32/50
22/22 [==============================] - 1s 31ms/step - loss: 0.7407 - acc: 0.9329 - val_loss: 1.6565 - val_acc: 0.5978
Epoch 33/50
22/22 [==============================] - 1s 31ms/step - loss: 0.7089 - acc: 0.9592 - val_loss: 1.7024 - val_acc: 0.6201
Epoch 34/50
22/22 [==============================] - 1s 34ms/step - loss: 0.6795 - acc: 0.9563 - val_loss: 1.7999 - val_acc: 0.5754
Epoch 35/50
22/22 [==============================] - 1s 31ms/step - loss: 0.6292 - acc:

## 3. Attention + BiLSTM + BERT

In [131]:
# ============================================
# 改进模型 2: Attention + BiLSTM + BERT (ziLSTM风格 + 正则化)
# ============================================
print("\n" + "=" * 50)
print("改进模型 2: Attention + BiLSTM + BERT (ziLSTM风格)")
print("=" * 50)

x = Input((82, 768))
h = Dropout(0.2)(x)  # 输入层 Dropout
h = Attention()([h, h])
h = Dropout(0.3)(h)  # Attention 后 Dropout
# ziLSTM 风格: BiLSTM(return_sequences=True) + GlobalMaxPooling1D
h = Bidirectional(LSTM(64, return_sequences=True, return_state=False,
                       kernel_regularizer=l2(0.01),
                       recurrent_regularizer=l2(0.01)))(h)
h = GlobalMaxPooling1D()(h)  # 提取最显著特征
h = Dropout(0.4)(h)
h = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(h)
h = Dropout(0.3)(h)
res = Dense(5, 'softmax', kernel_regularizer=l2(0.01))(h)

model_improved2 = Model(inputs=x, outputs=res)
model_improved2.compile(
    optimizer=Adam(learning_rate=0.0005),  # 降低学习率
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['acc']
)
model_improved2.summary()

# 回调函数
callback_list = [
    ModelCheckpoint(
        filepath="res2/T_model2_LSTM_bert.tf",
        monitor='val_loss',
        save_best_only=True,
        save_freq='epoch'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
]

# 训练
history2 = model_improved2.fit(
    x=np.asarray(bert_embs['train']),
    y=np.asarray(label['train']),
    batch_size=16,
    epochs=50,
    validation_data=[np.asarray(bert_embs['test']), np.asarray(label['test'])],
    callbacks=callback_list,
    class_weight=class_weight_dict,
    verbose=1
)

# 测试
model_improved2 = keras.models.load_model("res2/T_model2_LSTM_bert.tf")
pred = model_improved2.predict(np.asarray(bert_embs['test']))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])

eval_res = classification_report(
    numeric_test_labels, predicted_test_labels,
    target_names=['bored', 'happy', 'interested', 'tired', 'confusion'],
    digits=4, output_dict=False, zero_division=0
)
print(eval_res)
print('改进模型2 (Attention+BiLSTM+BERT ziLSTM风格):')
cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)


改进模型 2: Attention + BiLSTM + BERT (ziLSTM风格)
Model: "model_81"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_89 (InputLayer)          [(None, 82, 768)]    0           []                               
                                                                                                  
 dropout_267 (Dropout)          (None, 82, 768)      0           ['input_89[0][0]']               
                                                                                                  
 attention_23 (Attention)       (None, 82, 768)      0           ['dropout_267[0][0]',            
                                                                  'dropout_267[0][0]']            
                                                                                                  
 dropout_268 (Dropout)          (None, 82, 76

INFO:tensorflow:Assets written to: ./data2025/res2/T_model_improved2_BiLSTM_bert.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/T_model_improved2_BiLSTM_bert.tf/assets


22/22 [==============================] - 8s 311ms/step - loss: 11.4517 - acc: 0.1953 - val_loss: 10.6327 - val_acc: 0.1564
Epoch 2/50
21/22 [===========================>..] - ETA: 0s - loss: 10.1067 - acc: 0.2292

INFO:tensorflow:Assets written to: ./data2025/res2/T_model_improved2_BiLSTM_bert.tf/assets


...

INFO:tensorflow:Assets written to: ./data2025/res2/T_model_improved2_BiLSTM_bert.tf/assets


22/22 [==============================] - 6s 274ms/step - loss: 0.9247 - acc: 0.9446 - val_loss: 1.8118 - val_acc: 0.6257
Epoch 47/50
21/22 [===========================>..] - ETA: 0s - loss: 0.9579 - acc: 0.9256

INFO:tensorflow:Assets written to: ./data2025/res2/T_model_improved2_BiLSTM_bert.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/T_model_improved2_BiLSTM_bert.tf/assets


22/22 [==============================] - 5s 241ms/step - loss: 0.9564 - acc: 0.9242 - val_loss: 1.7560 - val_acc: 0.6592
Epoch 48/50
22/22 [==============================] - 1s 34ms/step - loss: 0.8834 - acc: 0.9534 - val_loss: 1.7608 - val_acc: 0.6257
Epoch 49/50
22/22 [==============================] - 1s 38ms/step - loss: 0.8519 - acc: 0.9592 - val_loss: 1.8410 - val_acc: 0.5754
Epoch 50/50
6/6 [==============================] - 0s 13ms/step
              precision    recall  f1-score   support

       bored     0.6471    0.8800    0.7458        50
       happy     0.7500    0.4444    0.5581        27
  interested     0.5385    0.7568    0.6292        37
       tired     0.9130    0.5526    0.6885        38
   confusion     0.6500    0.4815    0.5532        27

    accuracy                         0.6592       179
   macro avg     0.6997    0.6231    0.6350       179
weighted avg     0.6970    0.6592    0.6522       179

改进模型2 (Attention+BiLSTM+BERT ziLSTM风格):
[[44  1  3  1  1]
 [ 2

## BERT 系列结果对比

In [ ]:
# ============================================
# BERT 特征模型结果对比可视化
# ============================================
print("\n" + "=" * 50)
print("BERT 特征模型评估")
print("=" * 50)

# 加载模型
model_textcnn = keras.models.load_model('res2/T_TextCNN_bert.tf')
model_lstm = keras.models.load_model('res2/T_ziLSTM_bert.tf')
model_att_bilstm = keras.models.load_model('res2/T_model2_LSTM_bert.tf')

# 准备测试数据
bert_test = np.asarray(bert_embs['test'])
y_test = np.asarray(label['test'])

# 评估
test_loss_textcnn, test_acc_textcnn = model_textcnn.evaluate(bert_test, y_test, verbose=0)
test_loss_lstm, test_acc_lstm = model_lstm.evaluate(bert_test, y_test, verbose=0)
test_loss_att, test_acc_att = model_att_bilstm.evaluate(bert_test, y_test, verbose=0)

# 可视化对比
models_names = ['TextCNN', 'LSTM', 'Attention+BiLSTM']
accuracies = [test_acc_textcnn, test_acc_lstm, test_acc_att]

plt.figure(figsize=(10, 6))
plt.bar(models_names, accuracies, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.xlabel('Model', fontsize=12)
plt.ylabel('Test Accuracy', fontsize=12)
plt.title('BERT Feature Models Comparison', fontsize=14, fontweight='bold')
plt.ylim([0, 1])
for i, v in enumerate(accuracies):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom', fontsize=10)
plt.tight_layout()
plt.show()

print("\nBERT 特征模型结果：")
print(f"TextCNN: Test Accuracy = {test_acc_textcnn:.4f}, Test Loss = {test_loss_textcnn:.4f}")
print(f"LSTM: Test Accuracy = {test_acc_lstm:.4f}, Test Loss = {test_loss_lstm:.4f}")
print(f"Attention+BiLSTM: Test Accuracy = {test_acc_att:.4f}, Test Loss = {test_loss_att:.4f}")


# Baidubaike 特征实验

## 1. TextCNN + Baidu

In [116]:
# ============================================
# TextCNN + Baidu 特征
# ============================================
print("\n" + "=" * 50)
print("TextCNN + Baidu 特征")
print("=" * 50)

# 获取输入形状
baidu_train = np.asarray(baidu_embs['train'])
baidu_test = np.asarray(baidu_embs['test'])
input_shape_baidu = baidu_train.shape[1:]  # (seq_len, embedding_dim)
print(f"Baidu 输入形状: {input_shape_baidu}")

# 构建模型
model_cnn_baidu = build_textcnn(input_shape_baidu, num_classes=5)
model_cnn_baidu.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['acc']
)
model_cnn_baidu.summary()

# 回调函数
callback_list = [
    ModelCheckpoint(
        filepath="res2/T_TextCNN_baidu.tf",
        monitor='val_loss',
        save_best_only=True,
        save_freq='epoch'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
]

# 训练
history_cnn_baidu = model_cnn_baidu.fit(
    x=baidu_train,
    y=np.asarray(label['train']),
    batch_size=16,
    epochs=50,
    validation_data=[baidu_test, np.asarray(label['test'])],
    callbacks=callback_list,
    class_weight=class_weight_dict,
    verbose=1
)

# 测试
model_cnn_baidu = keras.models.load_model("res2/T_TextCNN_baidu.tf")
pred = model_cnn_baidu.predict(baidu_test)
predicted_labels = pred.argmax(axis=1)
numeric_labels = np.array(label['test'])

eval_res = classification_report(
    numeric_labels, predicted_labels,
    target_names=['bored', 'happy', 'interested', 'tired', 'confusion'],
    digits=4, output_dict=False, zero_division=0
)
print(eval_res)
print('TextCNN + Baidu:')
cm = confusion_matrix(y_true=numeric_labels.tolist(), y_pred=predicted_labels.tolist())
print(cm)


TextCNN + Baidu 特征
Baidu 输入形状: (82, 300)
Model: "model_66"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_74 (InputLayer)          [(None, 82, 300)]    0           []                               
                                                                                                  
 dropout_215 (Dropout)          (None, 82, 300)      0           ['input_74[0][0]']               
                                                                                                  
 conv1d_84 (Conv1D)             (None, 82, 128)      115328      ['dropout_215[0][0]']            
                                                                                                  
 conv1d_85 (Conv1D)             (None, 82, 128)      153728      ['dropout_215[0][0]']            
                                                 

INFO:tensorflow:Assets written to: ./data2025/res2/1T_TextCNN_baidu.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/1T_TextCNN_baidu.tf/assets


22/22 [==============================] - 1s 42ms/step - loss: 9.0467 - acc: 0.2653 - val_loss: 8.4403 - val_acc: 0.1620
Epoch 2/50
22/22 [==============================] - ETA: 0s - loss: 8.1306 - acc: 0.3236

INFO:tensorflow:Assets written to: ./data2025/res2/1T_TextCNN_baidu.tf/assets


...

INFO:tensorflow:Assets written to: ./data2025/res2/1T_TextCNN_baidu.tf/assets


22/22 [==============================] - 1s 36ms/step - loss: 0.7359 - acc: 0.9563 - val_loss: 1.6542 - val_acc: 0.5698
Epoch 45/50
22/22 [==============================] - 0s 10ms/step - loss: 0.6856 - acc: 0.9825 - val_loss: 1.6926 - val_acc: 0.5642
Epoch 46/50
22/22 [==============================] - 0s 10ms/step - loss: 0.6725 - acc: 0.9738 - val_loss: 1.8119 - val_acc: 0.5698
Epoch 47/50
17/22 [======================>.......] - ETA: 0s - loss: 0.6655 - acc: 0.9706

INFO:tensorflow:Assets written to: ./data2025/res2/1T_TextCNN_baidu.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/1T_TextCNN_baidu.tf/assets


22/22 [==============================] - 1s 35ms/step - loss: 0.6896 - acc: 0.9679 - val_loss: 1.6535 - val_acc: 0.5587
Epoch 48/50
22/22 [==============================] - 0s 10ms/step - loss: 0.6737 - acc: 0.9621 - val_loss: 1.6610 - val_acc: 0.5363
Epoch 49/50
22/22 [==============================] - 0s 10ms/step - loss: 0.6743 - acc: 0.9679 - val_loss: 1.7362 - val_acc: 0.5866
Epoch 50/50
6/6 [==============================] - 0s 5ms/step
              precision    recall  f1-score   support

       bored     0.5918    0.5800    0.5859        50
       happy     0.5000    0.7407    0.5970        27
  interested     0.4400    0.2973    0.3548        37
       tired     0.6667    0.6316    0.6486        38
   confusion     0.5517    0.5926    0.5714        27

    accuracy                         0.5587       179
   macro avg     0.5500    0.5684    0.5516       179
weighted avg     0.5564    0.5587    0.5509       179

TextCNN + Baidu:
[[29  3  5 10  3]
 [ 2 20  4  0  1]
 [ 3 16 11 

## 2. LSTM + Baidu

In [111]:
# ============================================
# ziBiLstm 风格 LSTM + Baidu 特征
# ============================================
print("\n" + "=" * 50)
print("ziBiLstm风格 LSTM + Baidu 特征")
print("=" * 50)

input_shape_baidu = np.asarray(baidu_embs['train']).shape[1:]
print(f"Baidu 输入形状: {input_shape_baidu}")

# 构建模型
model_zilstm_baidu = build_lstm_pooling(input_shape_baidu)
model_zilstm_baidu.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['acc']
)
model_zilstm_baidu.summary()

# 回调函数
callback_list = [
    ModelCheckpoint(
        filepath="res2/T_ziLSTM_baidu.tf",
        monitor='val_loss',
        save_best_only=True,
        save_freq='epoch'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
]

# 训练
history_zilstm_baidu = model_zilstm_baidu.fit(
    x=np.asarray(baidu_embs['train']),
    y=np.asarray(label['train']),
    batch_size=16,
    epochs=50,
    validation_data=[np.asarray(baidu_embs['test']), np.asarray(label['test'])],
    callbacks=callback_list,
    class_weight=class_weight_dict,
    verbose=1
)

# 测试
model_zilstm_baidu = keras.models.load_model("res2/T_ziLSTM_baidu.tf")
pred = model_zilstm_baidu.predict(np.asarray(baidu_embs['test']))
predicted_labels = pred.argmax(axis=1)
numeric_labels = np.array(label['test'])

eval_res = classification_report(
    numeric_labels, predicted_labels,
    target_names=['bored', 'happy', 'interested', 'tired', 'confusion'],
    digits=4, output_dict=False, zero_division=0
)
print(eval_res)
print('ziBiLstm风格 LSTM + Baidu:')
cm = confusion_matrix(y_true=numeric_labels.tolist(), y_pred=predicted_labels.tolist())
print(cm)


ziBiLstm风格 LSTM + Baidu 特征
Baidu 输入形状: (82, 300)
Model: "model_61"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_69 (InputLayer)       [(None, 82, 300)]         0         
                                                                 
 dropout_198 (Dropout)       (None, 82, 300)           0         
                                                                 
 lstm_35 (LSTM)              (None, 82, 128)           219648    
                                                                 
 global_max_pooling1d_113 (G  (None, 128)              0         
 lobalMaxPooling1D)                                              
                                                                 
 dropout_199 (Dropout)       (None, 128)               0         
                                                                 
 dense_122 (Dense)           (None, 128)               16512     
        

INFO:tensorflow:Assets written to: ./data2025/res2/1T_ziLSTM_baidu.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/1T_ziLSTM_baidu.tf/assets


22/22 [==============================] - 4s 151ms/step - loss: 6.8206 - acc: 0.2099 - val_loss: 5.5763 - val_acc: 0.3687
Epoch 2/50
20/22 [==========================>...] - ETA: 0s - loss: 4.9035 - acc: 0.2531

INFO:tensorflow:Assets written to: ./data2025/res2/1T_ziLSTM_baidu.tf/assets


...

INFO:tensorflow:Assets written to: ./data2025/res2/1T_ziLSTM_baidu.tf/assets


22/22 [==============================] - 2s 107ms/step - loss: 0.9079 - acc: 0.8601 - val_loss: 1.5699 - val_acc: 0.6089
Epoch 23/50
22/22 [==============================] - 1s 28ms/step - loss: 0.9385 - acc: 0.8455 - val_loss: 1.5979 - val_acc: 0.5698
Epoch 24/50
22/22 [==============================] - 1s 28ms/step - loss: 0.9358 - acc: 0.8338 - val_loss: 1.5712 - val_acc: 0.5754
Epoch 25/50
22/22 [==============================] - ETA: 0s - loss: 0.9571 - acc: 0.8280

INFO:tensorflow:Assets written to: ./data2025/res2/1T_ziLSTM_baidu.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/1T_ziLSTM_baidu.tf/assets


22/22 [==============================] - 2s 105ms/step - loss: 0.9571 - acc: 0.8280 - val_loss: 1.5549 - val_acc: 0.5754
Epoch 26/50
22/22 [==============================] - 1s 25ms/step - loss: 0.9053 - acc: 0.8513 - val_loss: 1.6483 - val_acc: 0.5642
Epoch 27/50
22/22 [==============================] - 1s 26ms/step - loss: 0.8546 - acc: 0.8513 - val_loss: 1.6955 - val_acc: 0.5363
Epoch 28/50
22/22 [==============================] - 1s 24ms/step - loss: 0.8149 - acc: 0.8980 - val_loss: 1.6087 - val_acc: 0.5642
Epoch 29/50
22/22 [==============================] - 1s 28ms/step - loss: 0.8524 - acc: 0.8601 - val_loss: 1.6779 - val_acc: 0.5419
Epoch 30/50
22/22 [==============================] - 1s 28ms/step - loss: 0.9525 - acc: 0.7959 - val_loss: 1.5697 - val_acc: 0.5642
Epoch 31/50
22/22 [==============================] - 1s 28ms/step - loss: 0.8491 - acc: 0.8571 - val_loss: 1.6178 - val_acc: 0.5642
Epoch 32/50
22/22 [==============================] - 1s 27ms/step - loss: 0.8215 - acc:

## 3. Attention + BiLSTM + Baidu

In [124]:
# ============================================
# Attention + BiLSTM + Baidu 特征 (ziLSTM风格)
# ============================================
print("\n" + "=" * 50)
print("Attention + BiLSTM + Baidu 特征 (ziLSTM风格)")
print("=" * 50)

# 构建模型
x = Input(input_shape_baidu)
h = Dropout(0.2)(x)
h = Attention()([h, h])
h = Dropout(0.3)(h)
# ziLSTM 风格: BiLSTM(return_sequences=True) + GlobalMaxPooling1D
h = Bidirectional(LSTM(64, return_sequences=True, return_state=False,
                       kernel_regularizer=l2(0.01),
                       recurrent_regularizer=l2(0.01)))(h)
h = GlobalMaxPooling1D()(h)  # 提取最显著特征
h = Dropout(0.4)(h)
h = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(h)
h = Dropout(0.3)(h)
res = Dense(5, 'softmax', kernel_regularizer=l2(0.01))(h)

model_att_bilstm_baidu = Model(inputs=x, outputs=res)
model_att_bilstm_baidu.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['acc']
)
model_att_bilstm_baidu.summary()

# 回调函数
callback_list = [
    ModelCheckpoint(
        filepath="res2/1T_Att_BiLSTM_baidu.tf",
        monitor='val_loss',
        save_best_only=True,
        save_freq='epoch'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
]

# 训练
history_att_bilstm_baidu = model_att_bilstm_baidu.fit(
    x=baidu_train,
    y=np.asarray(label['train']),
    batch_size=16,
    epochs=50,
    validation_data=[baidu_test, np.asarray(label['test'])],
    callbacks=callback_list,
    class_weight=class_weight_dict,
    verbose=1
)

# 测试
model_att_bilstm_baidu = keras.models.load_model("res2/1T_Att_BiLSTM_baidu.tf")
pred = model_att_bilstm_baidu.predict(baidu_test)
predicted_labels = pred.argmax(axis=1)
numeric_labels = np.array(label['test'])

eval_res = classification_report(
    numeric_labels, predicted_labels,
    target_names=['bored', 'happy', 'interested', 'tired', 'confusion'],
    digits=4, output_dict=False, zero_division=0
)
print(eval_res)
print('Attention + BiLSTM + Baidu (ziLSTM风格):')
cm = confusion_matrix(y_true=numeric_labels.tolist(), y_pred=predicted_labels.tolist())
print(cm)


Attention + BiLSTM + Baidu 特征 (ziLSTM风格)
Model: "model_74"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_82 (InputLayer)          [(None, 82, 300)]    0           []                               
                                                                                                  
 dropout_242 (Dropout)          (None, 82, 300)      0           ['input_82[0][0]']               
                                                                                                  
 attention_19 (Attention)       (None, 82, 300)      0           ['dropout_242[0][0]',            
                                                                  'dropout_242[0][0]']            
                                                                                                  
 dropout_243 (Dropout)          (None, 82, 300)  

INFO:tensorflow:Assets written to: ./data2025/res2/T_Att_BiLSTM_baidu.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/T_Att_BiLSTM_baidu.tf/assets


22/22 [==============================] - 8s 295ms/step - loss: 9.1177 - acc: 0.2303 - val_loss: 8.3733 - val_acc: 0.2235
Epoch 2/50
22/22 [==============================] - ETA: 0s - loss: 7.8355 - acc: 0.2391

INFO:tensorflow:Assets written to: ./data2025/res2/T_Att_BiLSTM_baidu.tf/assets


...

INFO:tensorflow:Assets written to: ./data2025/res2/T_Att_BiLSTM_baidu.tf/assets


22/22 [==============================] - 5s 233ms/step - loss: 1.1843 - acc: 0.7318 - val_loss: 1.6074 - val_acc: 0.5642
Epoch 46/50
22/22 [==============================] - 1s 26ms/step - loss: 1.1674 - acc: 0.7580 - val_loss: 1.7525 - val_acc: 0.5084
Epoch 47/50
22/22 [==============================] - ETA: 0s - loss: 1.1765 - acc: 0.7172

INFO:tensorflow:Assets written to: ./data2025/res2/T_Att_BiLSTM_baidu.tf/assets


INFO:tensorflow:Assets written to: ./data2025/res2/T_Att_BiLSTM_baidu.tf/assets


22/22 [==============================] - 5s 229ms/step - loss: 1.1765 - acc: 0.7172 - val_loss: 1.6041 - val_acc: 0.5642
Epoch 48/50
22/22 [==============================] - 1s 26ms/step - loss: 1.1220 - acc: 0.8017 - val_loss: 1.6617 - val_acc: 0.5196
Epoch 49/50
22/22 [==============================] - 1s 28ms/step - loss: 1.1351 - acc: 0.7697 - val_loss: 1.6887 - val_acc: 0.4804
Epoch 50/50
6/6 [==============================] - 0s 10ms/step
              precision    recall  f1-score   support

       bored     0.6000    0.6000    0.6000        50
       happy     0.6471    0.4074    0.5000        27
  interested     0.4808    0.6757    0.5618        37
       tired     0.5789    0.5789    0.5789        38
   confusion     0.5909    0.4815    0.5306        27

    accuracy                         0.5642       179
   macro avg     0.5795    0.5487    0.5543       179
weighted avg     0.5766    0.5642    0.5621       179

Attention + BiLSTM + Baidu (ziLSTM风格):
[[30  1  6 11  2]
 [ 2 

## Baidu 系列结果对比

In [ ]:
# ============================================
# Baidu 特征模型结果对比可视化
# ============================================
print("\n" + "=" * 50)
print("Baidu 特征模型评估")
print("=" * 50)

# 加载模型
model_textcnn = keras.models.load_model('res2/T_TextCNN_baidu.tf')
model_lstm = keras.models.load_model('res2/T_ziLSTM_baidu.tf')
model_att_bilstm = keras.models.load_model('res2/1T_Att_BiLSTM_baidu.tf')

# 准备测试数据
baidu_test = np.asarray(baidu_embs['test'])
y_test = np.asarray(label['test'])

# 评估
test_loss_textcnn, test_acc_textcnn = model_textcnn.evaluate(baidu_test, y_test, verbose=0)
test_loss_lstm, test_acc_lstm = model_lstm.evaluate(baidu_test, y_test, verbose=0)
test_loss_att, test_acc_att = model_att_bilstm.evaluate(baidu_test, y_test, verbose=0)

# 可视化对比
models_names = ['TextCNN', 'LSTM', 'Attention+BiLSTM']
accuracies = [test_acc_textcnn, test_acc_lstm, test_acc_att]

plt.figure(figsize=(10, 6))
plt.bar(models_names, accuracies, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.xlabel('Model', fontsize=12)
plt.ylabel('Test Accuracy', fontsize=12)
plt.title('Baidu Feature Models Comparison', fontsize=14, fontweight='bold')
plt.ylim([0, 1])
for i, v in enumerate(accuracies):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom', fontsize=10)
plt.tight_layout()
plt.show()

print("\nBaidu 特征模型结果：")
print(f"TextCNN: Test Accuracy = {test_acc_textcnn:.4f}, Test Loss = {test_loss_textcnn:.4f}")
print(f"LSTM: Test Accuracy = {test_acc_lstm:.4f}, Test Loss = {test_loss_lstm:.4f}")
print(f"Attention+BiLSTM: Test Accuracy = {test_acc_att:.4f}, Test Loss = {test_loss_att:.4f}")
